In [ ]:
# import librariesสำหรับอ่านเนื้อหาจากไฟล์pdf
import fitz
from langchain.docstore.document import Document
# ระบุpathที่อยู่ไฟล์เอกสาร
pdf1 = r"C:\Users\Kitti\OneDrive\เดสก์ท็อป\งาน\AI Engineer Test\AI Engineer Test\ai_test_user_feedback.pdf"
pdf2 = r"C:\Users\Kitti\OneDrive\เดสก์ท็อป\งาน\AI Engineer Test\AI Engineer Test\ai_test_bug_report.pdf"
# สร้างตัวแปรtextsไว้เก็บเนื้อหาจากเอกสาร
texts = []

for file_path in [pdf1, pdf2]:
    doc = fitz.open(file_path)
    for i, page in enumerate(doc): #วนลูปผ่านเอกสาร
        content = page.get_text()
        if content.strip():
            texts.append(Document(
                page_content=content,
                metadata={"source": file_path, "page": i + 1}
            ))

for i, doc in enumerate(texts[:3]):  # ทดลองดู3เอกสารแรก
    print(f"🔹 Document {i+1}")
    print(f"Source: {doc.metadata['source']} - Page: {doc.metadata['page']}") #ดูmetadataของเอกสาร
    print(doc.page_content[:300])  # แสดงเนื้อหาต้น 300 ตัวอักษร
    print("------")

In [ ]:
import os
from dotenv import load_dotenv

# load environment
load_dotenv(dotenv_path=r"C:\Users\Kitti\OneDrive\เดสก์ท็อป\งาน\Hugging Face\Tokens\HUGGINGFACE_API_KEY.env")

# ตั้งค่า API Key ของ Hugging Face
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY") # ใส่คีย์ API ของคุณที่นี่

if "HUGGINGFACE_API_KEY" in os.environ:
    print("Environment variable loaded successfully!")
    print("HUGGINGFACE_API_KEY:", os.environ["HUGGINGFACE_API_KEY"])  # ตรวจสอบค่า (อย่าแชร์ค่า API Key จริง)
else:
    print("Failed to load environment variable.")


In [ ]:
#import librariesที่จำเป็นสำหรับทำembeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# โหลดโมเดลสำหรับทำembeddingผ่านHuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# เก็บข้อมูลที่ถูกembeddingsแล้วไว้ในตัวแปรvector store
vectorstore = FAISS.from_documents(texts, embeddings)

# ใช้retrieverในการรับข้อมูลจากvector store
retriever = vectorstore.as_retriever()

In [ ]:
#import model LLM ที่ต้องการใช้งานเข้ามา
import ollama
#ทดลองใช้งานเบื้องต้น
prompt = "เมืองหลวงของประเทศไทยคือ"

response = ollama.chat(model="llama3.2", messages=[
    {"role": "system", "content": "You are male. Answer with นะครับ"},
    {"role": "user", "content": prompt}
])

response["message"]["content"]

In [ ]:
def generate_response(query_text):
     # ใช้retrieverดึงเอกสารที่เกี่ยวข้องกับ query_text
     docs = retriever.invoke(query_text)
     # สร้างcontextจากเนื้อหาของเอกสาร
     context = "\n".join([doc.page_content for doc in docs])
     # สร้างpromptสำหรับโมเดล
     prompt = f"""You are an internal AI assistant that helps the product and engineering team extract 
     insights from internal documents and team reports. use information from context to answer.
     if you can't find information that match to user's question, just say you don't find information.:\n{context}\nQuestion: {query_text}"""
     # ส่ง prompt เข้าโมเดล LLaMA ผ่าน Ollama
     response = ollama.chat(model="llama3.2", messages=[
        {"role": "system", "content": """You are an assistant that extracts structured insights from documents.
        You must always reply with a JSON object like this:
     {
      "Feedback #n": "...",
     {"Bug #n": 
       Title: "..."},
     }
     """},
        {"role": "user", "content": prompt}
     ])

     return response["message"]["content"]
# ทดลองเรียกใช้งาน     
print(generate_response("What are the issues reported on email notification?"))